In [1]:
import torch
import os
import numpy as np
import pandas as pd
from datasets import Dataset
from datasets import load_metric
from transformers import AutoTokenizer, DataCollatorWithPadding
from transformers import TrainingArguments, AutoModelForSequenceClassification, Trainer
from tqdm.notebook import tqdm
from torch.utils.data import DataLoader

from torchsummary import summary
tqdm.pandas()

DATAPATH = "data/data.csv"

In [2]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

C:\Users\PC\.virtualenvs\CMP711_Project\lib\site-packages\huggingface_hub\file_download.py:127: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\PC\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [3]:
def test_df(model, df, batch=128):
    def encodeText(example):
        return torch.tensor(tokenizer.encode(example, padding="max_length", truncation=True)).unsqueeze(0)
    df["text"] = df["text"].progress_apply(encodeText)
    vals = df["text"].values
    vals = np.vstack(vals).astype(np.int32)
    input_ids = torch.from_numpy(vals)
    print(input_ids.shape)
    feature_model = torch.nn.Sequential(*(list(model.children())[:-2]))

    feature_model.cuda()

    data = DataLoader(input_ids, batch_size=batch, shuffle=False)

    results = []
    for i, inp in enumerate(data):
        with torch.no_grad():
            last_hidden_states = feature_model(inp.cuda())[0][:, 0, :]
            results.append(last_hidden_states.cpu().numpy())

        print(last_hidden_states.shape)

    return results

In [4]:
clean_dataset = pd.read_csv(DATAPATH)
dataset = clean_dataset.copy()

In [5]:
embeddings = test_df(model, dataset)

  0%|          | 0/18045 [00:00<?, ?it/s]

torch.Size([18045, 512])
torch.Size([128, 768])
torch.Size([128, 768])
torch.Size([128, 768])
torch.Size([128, 768])
torch.Size([128, 768])
torch.Size([128, 768])
torch.Size([128, 768])
torch.Size([128, 768])
torch.Size([128, 768])
torch.Size([128, 768])
torch.Size([128, 768])
torch.Size([128, 768])
torch.Size([128, 768])
torch.Size([128, 768])
torch.Size([128, 768])
torch.Size([128, 768])
torch.Size([128, 768])
torch.Size([128, 768])
torch.Size([128, 768])
torch.Size([128, 768])
torch.Size([128, 768])
torch.Size([128, 768])
torch.Size([128, 768])
torch.Size([128, 768])
torch.Size([128, 768])
torch.Size([128, 768])
torch.Size([128, 768])
torch.Size([128, 768])
torch.Size([128, 768])
torch.Size([128, 768])
torch.Size([128, 768])
torch.Size([128, 768])
torch.Size([128, 768])
torch.Size([128, 768])
torch.Size([128, 768])
torch.Size([128, 768])
torch.Size([128, 768])
torch.Size([128, 768])
torch.Size([128, 768])
torch.Size([128, 768])
torch.Size([128, 768])
torch.Size([128, 768])
torch.Siz

In [7]:
emb_values = np.vstack(embeddings)
emb_values.shape

(18045, 768)

In [9]:
for i in range(emb_values.shape[1]):
    clean_dataset["f"+str(i+1)] = emb_values[:,i]

C:\Users\PC\AppData\Local\Temp\ipykernel_1824\2786580565.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  clean_dataset["f"+str(i+1)] = emb_values[:,i]
C:\Users\PC\AppData\Local\Temp\ipykernel_1824\2786580565.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  clean_dataset["f"+str(i+1)] = emb_values[:,i]
C:\Users\PC\AppData\Local\Temp\ipykernel_1824\2786580565.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joini

In [11]:
clean_dataset.to_csv("data/raw_bert_embeddings.csv", index=False)